Chengyu Dai, Cupjin Huang, Kevin Sung

generated by Jupyter notebook

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cvxopt import matrix, spmatrix, solvers
import networkx as nx
from networkx.algorithms import bipartite

import itertools
import json
import csv
import time as thyme

eps = 1e-10
solvers.options['show_progress'] = False

# Problem Statement:
In the generalized assignment problem, we are given a collection of $n$ jobs to be assigned to $m$ machines. Each job $j = 1, . . . , n$ is to be assigned to exactly one machine; if it is assigned to machine $i$, then it requires $p_{ij}$ time units of processing, and incurs a cost of $c_{ij}$. Furthermore, we are given a time bound $T$ that limits the total processing of each machine $i = 1, . . . , m$. The aim is to find a feasible assignment of minimum total cost.


- Input : 

 + number of jobs $n$,
 
 + number of machines $m$,
 
 + cost matrix $[c_{ij}]$,
 
 + processing time matrix $[p_{ij}]$,
 
 + time bound $T$.
 
- Output: $w\in [m]^n$, where $w_i$ is the machine where job $i$ is assigned to

## IP Formulation:
Denote $x_{ij}$ be the indicator of whether job $i$ is assigned to machine $j$. Then our problem can be formulated as the following IP:

\begin{equation*}
\begin{array}{ll@{}ll}
\text{minimize}  & \displaystyle\sum\limits_{i,j} x_{ij}&c_{ij} &\\
\text{subject to}& \displaystyle\sum\limits_{j=1}^m   &x_{ij} = 1,  &i=1 ,..., n\\
                  & \displaystyle\sum\limits_{i=1}^n   &p_{ij}x_{ij} \leq  T,  &j=1 ,\cdots, m\\
                 &                                                &x_{ij} \in \{0,1\}, &i=1,\cdots,n,\\
                 & & &j=1 ,\cdots, m
\end{array}
\end{equation*}

# Approximation Algorithm
The IP above cannot be solved both exactly and efficiently assuming $\mathcal{NP}\neq \mathcal{P}$. Here we will present an algorithm which gives a solution such that

- The cost is at most the optimal cost of the original problem;

- Each machine terminates before time $2T$.

The algorithm consists of three parts:

1. Solve the LP relaxation of the original IP

2. Construct a bipartite graph based on the fractional solution

3. Determinsticly round the fractional solution to integral with the help of the bipartite graph 

In [2]:
def genAssign(p,c,T, show=False):
    '''
    Generate an assignment with maximum completion time <= 2T and total cost <= LP.
    Input:
        p: processing time matrix with dimension mxn
        c: cost matrix with dimension mxn
        T: list of time bounds (in the original problem these are all the same)
        show: whether intermediate results are to be printed
    Output:
        res: an array indicating the assignment
        f: the LP value
        s: LP solver return status
        t1 - t0: running time of LP
        t3 - t2: running time of rounding
    '''
    t0 = thyme.perf_counter()
    x,f,s = getLPSol(p,c,T)
    t1 = thyme.perf_counter()
    if x is None:
        return None, f, s, t1 - t0, np.nan
    t2 = thyme.perf_counter()
    B = buildGraph(x,p)
    res = detRound(B, c,show)
    t3 = thyme.perf_counter()
    return res, f, s, t1 - t0, t3 - t2

## LP Relaxation


Consider the following LP relaxation:

\begin{equation*}
\begin{array}{ll@{}ll}
\text{minimize}  & \displaystyle\sum\limits_{i,j} x_{ij}&c_{ij} &\\
\text{subject to}& \displaystyle\sum\limits_{j=1}^m   &x_{ij} = 1,  &i=1 ,..., n\\
                  & \displaystyle\sum\limits_{i=1}^n   &p_{ij}x_{ij} \leq  T,  &j=1 ,\cdots, m\\
                 &                                                &x_{ij} \geq 0, &i=1,\cdots,n,\\
                 & & &j=1 ,\cdots, m
\end{array}
\end{equation*}

Solving this LP gives us a fractional solution $\mathbf{x}\in [0,1]^{n\times m}$.

In [3]:
def getLPSol(p, c, T):
    """
    Get a solution to the LP. There are n jobs and m machines
    Inputs:
        p: an mxn matrix
        c: an mxn matrix
        T: a list of size m
    Output: an mxn matrix
    """
    # get LP dimensions
    m, n = p.shape

    d = np.empty(m * n)                                 # coefficient vector
    A_ub = []                                           # upper bound constraint matrix (sparse)
    b_ub = np.concatenate((T, np.zeros(2 * m * n)))     # upper bound constraint vector
    A_eq = np.ones(m * n)                               # equality constraint matrix (sparse)
    b_eq = np.ones(n)                                   # equality constraint vector

    # indices for sparse data types
    A_ubI = []
    A_ubJ = []
    A_eqI = []
    A_eqJ = []

    for i in range(m):
        for j in range(n):
            # construct coefficient vector
            d[i * n + j] = c[i, j]
            # construct equality constraint matrix
            A_eqI.append(j)
            A_eqJ.append(i * n + j)
            # construct entries to upper bound matrix
            A_ub.append(p[i, j])
            A_ubI.append(i)
            A_ubJ.append(i * n + j)
            A_ub.append(-1)
            A_ubI.append(m + i * n + j)
            A_ubJ.append(i * n + j)
            if p[i, j] > T[i]:
                A_ub.append(1)
                A_ubI.append(m + m * n + i * n + j)
                A_ubJ.append(i * n + j)

    e = matrix(d)
    G = spmatrix(A_ub, A_ubI, A_ubJ, size=(m + 2 * m * n, m * n))
    h = matrix(b_ub)
    A = spmatrix(A_eq, A_eqI, A_eqJ, size=(n, m * n))
    b = matrix(b_eq)

    # get LP solution
    sol = solvers.lp(e, G, h, A, b, solver='glpk')
    if sol['status'] == 'primal infeasible':
        return None, np.nan, sol['status']
    return np.reshape(np.array(sol['x']), (m, n)), sol['primal objective'], sol['status']


## Constructing Bipartite Graph

Let $k_j = \lceil\sum_{i=1}^n x_{ij}\rceil$. Define a bipartite graph $(A,B)$, where $A=[n]$ and $B=\{(j,s)|j\in[m], s\in[s_j]\}$. We illustrate the edges in the bipartite graph by only looking at the case $j=1$.

For $j=1$, consider only the jobs $i$ such that $x_{i1}>0$ (assume this is $[n]$ wlog). Assume that $p_{11}\geq p_{21}\geq\cdots p_{n1}$ without loss of generality (sort them if necessary). For each $i'$, we track the partial sum $\sum_{i=1}^{i'}x_{i1}$.

- If $\sum_{i=1}^{i'}x_{i1}\leq s, \sum_{i=1}^{i'-1}x_{i1}\geq s-1$, then add an edge $(i,(1,s))$ with weight $x_{i'1}$;
- If $\sum_{i=1}^{i'}x_{i1}> s, \sum_{i=1}^{i'-1}x_{i1}< s-1$, then add an edge $(i,(1,s-1))$ with weight $s-\sum_{i=1}^{i'-1}x_{i1}$ and an edge $(i,(1,s))$ with weight $\sum_{i=1}^{i'}x_{i1}-s$.

In [4]:
def buildGraph(x, p,):
    """
    Convert an LP solution to a bipartite graph with fractional
    edge weights where one side represents jobs and the other side
    represents slots
    Inputs:
        x: an mxn matrix
        p: an mxn matrix. These are the job completion times.
        It's required because we need to sort the rows
        of x based on the entries of p.
    Output: a nxmxn array B, where B[j, i, s] represents the weight
    on the edge from job j to slot s of machine i.
    """
    m = x.shape[0]
    n = x.shape[1]

    B = np.zeros((n, m, n))

    for i in range(m):
        s = 0       # current slot in machine i
        left = 1    # space left in current slot
        x_indices = np.argsort(-p[i]) # indices of x when p_i is in nondecreasing order
        for j in x_indices:
            y = x[i, j]
            if y <= left:
                # we can fit this job into slot s
                B[j, i, s] = y
                left -= y
            else:
                # we need to split this job into two slots
                B[j, i, s] = left       # fill up current slot
                s += 1                  # increment slot
                B[j, i, s] = y - left   # put remaining weight in new slot
                left = 1 - (y - left)   # update space left in new slot

    return B

## Deterministic Rounding

Note that the bipartite graph is a fractional min-cost perfect matching of the underlying cost function $C(i,(j,s))=c_{ij}$. It can be shown that all extreme points of the polytope are integral, and there is a way to convert the fractional solution into an integral one within polynomial time, without increasing the objective value.

Call all edges with weight in the range $(0,1)$ _unsaturated_ and $\{0,1\}$ _saturated_. We present an iterative method such that after one loop, the number of unsaturated edges decreases by at least $1$, and the sum of weights attached to a left vertex is preserved as $1$. Since at the end there is no unsaturated edge left and each left vertex is attached to edges with weights summing up to $1$, each left vertex is mapped to a right vertex via the unique edge with weight $1$. We will see shortly that this map is also injective, proving that it is a perfect matching on the left side.

Consider the following two cases:

1. There is a cycle of unsaturated edges. Color them by blue and red alternately. By increasing the weights of all red edges by a small amount $\delta$ and decreasing the weights of blue ones by $\delta$, no LP constraints are violated provided that $\delta$ is sufficiently small. As the objective function is linear in $\delta$, there must be one direction where the objective value is not increased. We can thus move $\delta$ towards that direction until one of the red edges comes to $1$ or one of the blue edges comes to $0$. Either way we decreased the number of unsaturated edges by at least $1$.

2. Call a vertex _single_ if it is only attached to an unsaturated edge, then the weight on the vertex is easily seen as equal to the the weight of the edge. Note that no left vertex can be single since they all have weight exactly $1$. Suppose now there is no cycle of unsaturated edge left, then each maximum path of unsaturated edge must start and end at single vertices. Choose a maximum path and color the edges by blue and red alternatively. Similar to case 1, perturbing the weight of all blue and red edges by a small amount $\delta$ does not violate any of the LP constraints, and we can freely move $\delta$ until one red edge reaches $1$ or one blue edge reaches $0$. 

In [5]:
def detRound(X, C,show=False):
    '''
    Deterministicly round the input fractional min-cost perfect matching to a integral one and output it.
        Input:
            X: fractional perfect matching represented by adjacency matrix
            C: the cost matrix
        Output:
            res: result of the assignment
    '''
    n,m,k = X.shape
    res = np.ones(n,dtype='int')*(-1)
    B = np.reshape(X, (X.shape[0],m*k))
    
    # Preprocessing - get rid of all edges w/ weight 1
    for i in range(n):
        for j in range(m*k):
            if(B[i,j]>1-eps):
                res[i] = j//k
                B[i,j] = 0

    while(list(res).count(-1) != 0): # not all jobs are assigned
        cyc = cycle_bipartite(B)  # finding a cycle or augmenting path
        
        #draw(B, cyc, show=show)
        pairs = round_cycle(B, cyc,k,C) # round the cycle
        if(pairs.shape[0] != 0): #  record all fulfilled edges
            for j in range(pairs.shape[0]):
                res[pairs[j,0]]= pairs[j,1] //k
                B[pairs[j,0]][pairs[j,1]]=0
        
    return res

In [6]:
def cycle_bipartite(x):
    '''
    Find a cycle or an augmenting path in a bipartite graph.
        Input: 
            x: a bipartite graph encoded in an (n,m) matrix
        Output:
            cyc: indices of a cycle, where there is an edge cyc[i][0]->cyc[i][1] and cyc[i][1]->cyc[i+1][0]
        
    '''
    cl = []
    cr = []
    st = np.where(np.sum(x, axis = 0)> eps)[0]
    xcopy = np.copy(x)
    res = dfs_right(xcopy,cl,cr,st)
    if(res is None):
        print(xcopy)
    return res
    
def dfs_left(x,cl,cr, l):
    cl.append(-1)
    for i in range(len(l)):
        cl[-1]=l[i]          
        s = cl[-1]
        t = cr[-1]
        x[s,t]=0
        llist = np.where(x[cl[-1],:] > eps)[0]
        if(cr[0] in llist):
            cr.append(cr[0])
            cyc = [cl, cr]
            return cyc
        cyc = dfs_right(x, cl, cr, llist)
        if(cyc != None):
            return cyc   
        if(len(cr)!= 0):
            x[s,t]=1
    cl.pop()
    return None

def dfs_right(x,cl,cr, l):  
    cr.append(-1)
    for i in range(len(l)):
        cr[-1]=l[i]
        s = -1
        t = cr[-1]
        if(len(cl)>0):
            s = cl[-1]
            x[s,t]=0
        rlist = np.where(x[:,t] >eps)[0]
        if(rlist.shape[0] == 0):
            if(np.where(x[:,cr[0]]>eps)[0].shape[0] == 0):
                cyc = [cl, cr]
                return cyc
        cyc = dfs_left(x, cl, cr, rlist)
        if(cyc != None):
            return cyc        
        if(len(cl)>0):
            x[s,t]=1
    cr.pop()
    return None

In [7]:
def round_cycle(B, cyc,k,C):
    '''
        Input:
            B: an adjacency matrix
            cyc: a cycle in B
            k: the number of slots for each machine
            C: the cost matrix
        Output:
            pairs: lists of edges with weight 1
            B is rounded such that at least one edge in the cycle in the graph is saturated
    '''
    
    slope = 0 # determine which direction to round
    l = len(cyc[0])
    for i in range(l):
        slope += C[cyc[1][i]// k][cyc[0][i]]-C[cyc[1][i+1]// k][cyc[0][i]]
    if(slope > eps): # flip the cycle to ensure the LP value is not increased
        cyc[0] = cyc[0][::-1]
        cyc[1] = cyc[1][::-1]
        


    w = np.amin([1-B[cyc[0][i],cyc[1][i]] for i in range(l)])
    d = np.amin([B[cyc[0][i],cyc[1][i+1]] for i in range(l)])
    c = min([w,d])
            
    pairs = []
    curr = cyc[1][0]
    for i in range(l):

        # increase the weights of red edges
        curl = cyc[0][i]
        B[curl][curr] +=  c
        if(B[curl][curr] > 1-eps): # new integral edge found
            pairs.append([curl,curr])
        # decrease the weights of blue edges
        curr = cyc[1][i+1]
        B[curl][curr] -=  c
        if(B[curl][curr] < eps):
            B[curl][curr] = 0
    return np.array(pairs)

# Test

## Correctness Test

In [8]:
def cost(res, C):
    r = 0
    for i in range(len(res)):
        if(res[i]>= 0):
            r += C[res[i],i]
    return r
def time(res, P):
    r = np.zeros((P.shape[0]))
    for i in range(len(res)):
        r[res[i]]+= P[res[i],i]
    return r
def cost_B(B,C,n,m):
    r = 0
    for i in range(n):
        for j in range(m):
            for k in range(n):
                r += C[j,i] * B[i,j*n+k]
    return r
    

In [9]:
flatness_c = 3
flatness_p = 2
n = 20
m = 5


for i in range(100):
    C = np.exp(flatness_c*np.random.rand(m,n))
    P = np.exp(flatness_p*np.random.rand(m,n))
    T = np.sum(P)*np.random.rand() * 0.05 * np.ones(m)
    res, f, s, tLP, tRound = genAssign(P,C,T)
    if(res is None):
        if s == 'primal infeasible':
            print("Problem appears to be infeasible.")
        if s == 'unknown':
            print("Status unknown.")
    else:
        if s == 'unknown':
            print("Terminated early.")
        print(res,f, cost(res, C), T[0], np.amax(time(res, P)))

[0 2 0 3 3 1 0 2 3 4 2 1 0 0 4 2 4 1 1 1] 39.42277596607139 39.34869888869089 14.791933156717352 16.54940737010041
[4 3 0 1 3 4 3 3 4 1 0 0 2 0 1 2 4 1 2 2] 73.08197940389245 51.46081193888764 6.623861477162112 9.53907884634175
Problem appears to be infeasible.
Problem appears to be infeasible.
[2 0 2 0 3 4 3 1 2 3 1 4 3 4 4 1 2 2 4 3] 42.17934342480293 41.21618126713864 10.510596327439053 15.151386249113344
[1 2 3 1 4 0 4 3 2 1 0 1 0 2 2 3 4 1 3 0] 38.27052765487596 37.73056774129891 11.26814028311986 13.97026039709148
Problem appears to be infeasible.
[2 3 1 1 2 1 0 4 2 2 0 4 3 0 0 3 4 3 1 4] 51.52731452960801 48.45263826475952 7.382139483219477 10.256164139781376
[2 1 3 3 3 3 4 4 2 4 1 2 1 3 0 4 2 0 3 2] 31.06295965622452 30.88036921670472 13.301969729771296 18.779161979989947
[0 4 4 2 3 2 2 1 3 2 3 0 0 1 0 1 1 4 3 2] 45.922614622606304 40.880712179755704 8.655046521610442 10.984023305166287
[3 3 4 1 1 3 4 0 1 0 3 4 1 2 4 3 0 2 2 2] 40.65660814849843 37.10825623765844 11.51111546107

## Benchmark tests

In [10]:
# data taken from http://astarte.csr.unibo.it/gapdata/gapinstances.html
def readjson(f):
    """ Convert json to problem instance
    Input:
        f: file object of json file
    Output:
        (p, c, T) tuple
    """
    a = json.load(f)
    p = np.array(a['req'], dtype='float')
    c = np.array(a['cost'], dtype='float')
    T = np.array(a['cap'], dtype='float')
    return p, c, T

# get data table of ids, LP vals, and opt vals and/or UB
data = {}
ids = []
with open('data/data.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        idee = row[0]
        ids.append(idee)
        data[idee] = {'n': row[1], 'm': row[2], 'LP': row[3], 'opt': row[4], 'UB': row[5]}

In [ ]:
# Run benchmark tests

"""
After this loop, 'data' will be a dict of dicts.
The keys for the outer dict are the ids of the problem instances
in the benchmark data. The inner dicts will have the following keys:
n, m, LP, opt, UB, algLP, algcost, algT, T, status, time
"""

for idee in ids:
    print(idee)
    d = data[idee]
    P,C,T = readjson(open('data/' + idee + '.json'))
    res, f, s, tLP, tRound = genAssign(P,C,T)
    d['timeLP'] = tLP
    d['timeRound'] = tRound
    d['status'] = s
    if(res is None):
        d['algLP'] = np.nan
        d['algcost'] = np.nan
        d['algT'] = [np.nan]
        d['T'] = [np.nan]
        if s == 'primal infeasible':
            print("Problem appears to be infeasible.")
        if s == 'unknown':
            print("Status unknown.")
    else:
        if s == 'unknown':
            print("Terminated early.")
        d['algLP'] = f
        d['algcost'] = cost(res, C)
        d['algT'] = time(res,P)
        d['T'] = T
        print('\t', d['algcost'], d['opt'], d['algLP'], d['LP'])
        print('\t', d['algT'])
        print('\t', d['T'])
        # print list of ratios of times
        print('\t', [format(d['algT'][i]/d['T'][i], '.2f') for i in range(len(d['algT']))])

gapa_0
	 1696.0 1698 1697.7272727272725 1697.73
	 [267. 301. 282. 318. 361.]
	 [342. 342. 342. 342. 342.]
	 ['0.78', '0.88', '0.82', '0.93', '1.06']
gapa_1
	 3234.0 3235 3234.739130434782 3234.74
	 [688. 612. 523. 534. 623.]
	 [671. 671. 671. 671. 671.]
	 ['1.03', '0.91', '0.78', '0.80', '0.93']
gapa_2
	 1358.0 1360 1358.556923076923 1358.56
	 [192. 146. 202. 131.  95. 195. 112. 192. 184.  66.]
	 [192. 192. 192. 192. 192. 192. 192. 192. 192. 192.]
	 ['1.00', '0.76', '1.05', '0.68', '0.49', '1.02', '0.58', '1.00', '0.96', '0.34']
gapa_3
	 2623.0 2623 2623.0 2623
	 [245. 387. 351. 326. 239. 283. 365. 279. 266. 274.]
	 [365. 365. 365. 365. 365. 365. 365. 365. 365. 365.]
	 ['0.67', '1.06', '0.96', '0.89', '0.65', '0.78', '1.00', '0.76', '0.73', '0.75']
gapa_4
	 1157.0 1158 1157.08 1157.08
	 [ 20. 104. 109.  91.  78.  62.  87.  31. 102.  98.  47.  42.  95.  84.
   0. 103.  89.  99.  90.  57.]
	 [100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 

In [ ]:
# write data
with open('benchmark_data_glpk3.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', 'n', 'm', 'status', 'LP', 'algLP', 'opt', 'UB', 'algcost', 'algmaxTratio', 'timeLP', 'timeRound'])
    for idee in ids:
        d = data[idee]
        writer.writerow([idee, d['n'], d['m'], d['status'], d['LP'], d['algLP'], d['opt'], d['UB'], d['algcost'], max([d['algT'][i]/d['T'][i] for i in range(len(d['algT']))]), d['timeLP'], d['timeRound']])

## graph viz for cycle finding and rounding process

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import itertools

def draw(x, cycle, show=False):
    if (show==False):
        return
#     ans = {j:i for i,j in enumerate(cycle_bipartite(x, self.start, debug=1))}
#     print(cycle)
    cycle_edges = []
    for idx, j in enumerate(cycle[-1]):
        if idx != 0:
            cycle_edges.append(("i{}".format(cycle[0][idx-1]), "j{}".format(j)))
        if idx != len(cycle[-1])-1:
            cycle_edges.append(("j{}".format(j),"i{}".format(cycle[0][idx])))

    B = nx.Graph()
    B.add_nodes_from(['i{}'.format(i) for i in range(x.shape[0])], bipartite=0)
    B.add_nodes_from(['j{}'.format(i) for i in range(x.shape[1])], bipartite=1)
    B.add_edges_from([('i{}'.format(i),'j{}'.format(j))
                      for i,j in itertools.product(range(x.shape[0]), range(x.shape[1]))
                      if x[i,j]
                     ])
    
    edge_color = []
    for edge in B.edges():
        if edge not in cycle_edges and (edge[-1],edge[0]) not in cycle_edges:
            edge_color.append(-100)
        else:
            try:
                idx = cycle_edges.index(edge)
            except:
                idx = cycle_edges.index((edge[-1],edge[0]))
            edge_color.append(idx%2)
            
    X = set(n for n,d in B.nodes(data=True) if d['bipartite']==0)
    Y = set(B) - X
    pos = dict()
    pos.update( (n, (1, int(n[1:])*60)) for i, n in enumerate(X) ) 
    pos.update( (n, (2, int(n[1:])*30)) for i, n in enumerate(Y) ) 
    
#     print(B.edges())
#     print(cycle_edges)
#     print(edge_color)
    nx.draw_networkx(B, pos=pos, with_labels=1, node_size=400,edge_color=edge_color, 
                     edge_cmap=plt.get_cmap('Paired'), edge_vmin=-1, edge_vmax=1, width=4)
#             node_color=[ans[n] if n in ans else '0' for n in B.nodes()],)
    plt.show()


In [ ]:
flatness_c = 3
flatness_p = 2
n = 5
m = 2

from tqdm import *

# good seed: 3

for i in tqdm([78]):
    np.random.seed(i)
    C = np.exp(flatness_c*np.random.rand(m,n))
    P = np.exp(flatness_p*np.random.rand(m,n))
    T = np.sum(P)*np.random.rand()
    res, f = genAssign(P,C,T, show=True)
    if(res == None):
        print("There is no feasible solution!")
    else:
        print(res,f, cost(res, C), T, time(res, P))

## performance scaling

In [ ]:
flatness_c = 3
flatness_p = 2

from tqdm import *

np.random.seed(0)

def perf(n,m,flatness_c,flatness_p):
    for i in (range(1)):
        C = np.exp(flatness_c*np.random.rand(m,n))
        P = np.exp(flatness_p*np.random.rand(m,n))
        T = np.sum(P)*np.random.rand()
        res, f = genAssign(P,C,T)

%timeit perf(10,5,flatness_c,flatness_p)
%timeit perf(50,5,flatness_c,flatness_p)
%timeit perf(100,5,flatness_c,flatness_p)
%timeit perf(300,5,flatness_c,flatness_p)
%timeit perf(500,5,flatness_c,flatness_p)
%timeit perf(1000,5,flatness_c,flatness_p)
# %timeit perf(5000,20,flatness_c,flatness_p)

In [ ]:
df = pd.DataFrame.from_dict({10:0.00954, 50:0.0567, 100:0.159, 300:1.23, 500:2.05, 1000:6.27},orient='index')
df.sort(0,inplace=1)
df.plot()

In [ ]:
flatness_c = 3
flatness_p = 2

from tqdm import *

np.random.seed(0)

def perf(n,m,flatness_c,flatness_p):
    for i in (range(1)):
        C = np.exp(flatness_c*np.random.rand(m,n))
        P = np.exp(flatness_p*np.random.rand(m,n))
        T = np.sum(P)*np.random.rand()
        res, f = genAssign(P,C,T)


%timeit perf(500,2,flatness_c,flatness_p)
%timeit perf(500,5,flatness_c,flatness_p)
%timeit perf(500,10,flatness_c,flatness_p)
%timeit perf(500,20,flatness_c,flatness_p)
%timeit perf(500,30,flatness_c,flatness_p)
%timeit perf(500,50,flatness_c,flatness_p)
%timeit perf(500,100,flatness_c,flatness_p)
%timeit perf(500,500,flatness_c,flatness_p)


1 loop, best of 3: 1.57 s per loop

1 loop, best of 3: 2.11 s per loop

1 loop, best of 3: 3.17 s per loop

1 loop, best of 3: 5.55 s per loop

1 loop, best of 3: 8.24 s per loop

1 loop, best of 3: 15 s per loop

1 loop, best of 3: 34.2 s per loop

In [ ]:
dic={2:1.57,5:2.11,10:3.17,20:5.55,30:8.24,50:15,100:34.2}
df = pd.DataFrame.from_dict(dic,orient='index')
df.sort(0,inplace=1)
df.plot()